In [119]:
import sys
from typing import Literal

sys.path.append("../../climateguard")
import pandas as pd
import numpy as np
from tqdm import tqdm
from llama_index.llms.ollama import Ollama
from pydantic import BaseModel, Field
from transformers import pipeline
from utils.llamaindex import get_pydantic_program

In [24]:
qwen = Ollama(model="qwen2.5:14b")
llama = Ollama(model="llama3.1:latest")

In [25]:
pipe = pipeline("text-classification", model="climatebert/environmental-claims", device=0)


In [26]:
environmental_claims = pd.read_csv("data/environmental_claims_train.csv")

In [27]:
%timeit -n 1
results = pipe(environmental_claims.text.to_list())

In [28]:
results = pd.DataFrame(results)

In [29]:
results.label = np.where(results.label == "yes", 1, 0)

In [30]:
results["positive_score"] = np.where(
    results.label == 1,
    results.score.values,
    1 - results.score.values,
)


In [31]:
from sklearn.metrics import classification_report, precision_recall_curve

In [32]:
print(classification_report(environmental_claims.label, results.label))


              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1585
           1       0.88      0.67      0.76       532

    accuracy                           0.89      2117
   macro avg       0.89      0.82      0.85      2117
weighted avg       0.89      0.89      0.89      2117



In [33]:
precision, recall, thresholds = precision_recall_curve(environmental_claims.label, results.positive_score)


In [34]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=recall, y=precision, mode="lines", name="P-R Curve"))
no_skill = len(environmental_claims.label[environmental_claims.label==1]) / len(environmental_claims.label)
fig.add_trace(go.Scatter(x=np.linspace(0, 1, len(precision)), y=np.ones_like(precision) * no_skill, mode="lines", name="No Skill"))
fig.update_layout(title="Precision-Recall Curve", xaxis_title="Recall", yaxis_title="Precision")
fig.show()


In [37]:
from huggingface_hub import login
from dotenv import load_dotenv
import os
load_dotenv("../../.env")


True

In [38]:
login(token=os.getenv("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/giuseppeguarino/.cache/huggingface/token
Login successful


In [162]:
from llama_cpp import Llama

llm = Llama(
	model_path="../../models/gemma_2_7b_q5_k_s/gemma-7b-it.Q4_K_S-v2.gguf",
    chat_format="gemma",
    verbose=False
)

In [163]:
response = llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

In [164]:
from llama_index.core.prompts import PromptTemplate

In [165]:
output_format = """{"label": "<label>", "explanation": "<explanation>"}"""

prompt = """You are a classifier that determines if a claim is environmental or not.

Task:
- Classify if the following claim is environmental (1) or not environmental (0)
- Provide a brief explanation for your classification
- Return ONLY a JSON object with no additional text

Rules:
- label must be either "1" (environmental) or "0" (not environmental)
- explanation must be a single sentence
- response must be valid JSON
- do not add any text before or after the JSON

Format your response exactly like this:
{output_format}

Example claim: "Our company reduced carbon emissions by 50%"
Example response:
{"label": "1", "explanation": "The claim directly addresses environmental impact through carbon emission reduction"}

Now classify this claim:
{claim}

Remember:
- Return ONLY the JSON object
- No additional text before or after
- Must be valid JSON format
"""

class ClaimLabel(BaseModel):
    label: Literal[1, 0, "1", "0"] = Field(description="The label of the claim. 1 for environmental and 0 for not environmental.")
    explanation: str = Field(description="1 sentence justifying your rating in FRENCH")

In [166]:
import json

prompt_template = PromptTemplate(template=prompt)
response = llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": prompt_template.format(output_format=output_format, claim=environmental_claims.text.to_list()[0])
		}
	]
)
text = response['choices'][0]['message']['content']
print(text)
ClaimLabel(**json.loads(text))

{"label": "1", "explanation": "The claim relates to the development of hydrogen strategies, which can contribute to climate targets"}


ClaimLabel(label='1', explanation='The claim relates to the development of hydrogen strategies, which can contribute to climate targets')

In [167]:
def classify_claim(llm: Llama, prompt_template: PromptTemplate, **kwargs) -> ClaimLabel:
    response = llm.create_chat_completion(
        messages = [
            {
                "role": "user",
                "content": prompt_template.format(**kwargs)
            }
        ]
    )
    text = response['choices'][0]['message']['content']
    try:
        return ClaimLabel(**json.loads(text))
    except Exception as e:
        print(e, text)
        raise e

classify_claim(llm, prompt_template, output_format=output_format, claim=environmental_claims.text.to_list()[0])

ClaimLabel(label='1', explanation='The claim relates to the development of hydrogen strategy, which has environmental implications')

In [168]:
test_cases = environmental_claims.text.to_list()

In [169]:
predictions = []
skipped = []
for idx, claim in tqdm(enumerate(test_cases), total=len(test_cases)):
    try:
        predictions.append(classify_claim(llm, prompt_template, output_format=output_format, claim=claim))
    except Exception as e:
        print(e, )
        skipped.append(idx)


  1%|          | 12/2117 [00:30<1:37:49,  2.79s/it]

Extra data: line 1 column 90 (char 89) {"label": "0", "explanation": "The claim does not directly address environmental issues"}}
Extra data: line 1 column 90 (char 89)


  2%|▏         | 41/2117 [01:51<1:50:20,  3.19s/it]

Extra data: line 1 column 90 (char 89) {"label": "0", "explanation": "The claim does not directly address environmental issues"}}

**Explanation:**

Although the text mentions expansion and cost competitiveness, it does not relate to environmental concerns. The focus is on business expansion and cost competitiveness, not environmental impact.
Extra data: line 1 column 90 (char 89)


  2%|▏         | 46/2117 [02:05<1:44:15,  3.02s/it]

...

 98%|█████████▊| 2084/2117 [13:01:07<01:46,  3.22s/it]

Extra data: line 1 column 112 (char 111) {"label": "1", "explanation": "The claim relates to environmental impact through a reduction in CO2 emissions"}}
Extra data: line 1 column 112 (char 111)


 99%|█████████▉| 2094/2117 [13:01:41<01:17,  3.35s/it]

Extra data: line 1 column 128 (char 127) {"label": "1", "explanation": "The claim relates to environmental impact through emission quotas and carbon dioxide emissions"}}
Extra data: line 1 column 128 (char 127)


100%|██████████| 2117/2117 [13:02:58<00:00, 22.19s/it]


In [174]:
skipped

[11,
 40,
 45,
 59,
 79,
 117,
 133,
 148,
 177,
 199,
 228,
 321,
 324,
 430,
 490,
 532,
 633,
 653,
 672,
 729,
 791,
 795,
 831,
 957,
 991,
 1041,
 1092,
 1124,
 1144,
 1195,
 1222,
 1241,
 1248,
 1249,
 1271,
 1286,
 1417,
 1478,
 1498,
 1509,
 1529,
 1595,
 1610,
 1612,
 1723,
 1739,
 1837,
 1866,
 1888,
 1960,
 1961,
 1979,
 1996,
 2049,
 2083,
 2093]

In [177]:
ground_truth = environmental_claims.label.to_list()
for skip_idx in reversed(skipped):
    ground_truth.pop(skip_idx)


In [179]:
predictions_arr = np.array([pred.label for pred in predictions]).astype(int)
print(classification_report(ground_truth, predictions_arr))



              precision    recall  f1-score   support

           0       0.94      0.84      0.88      1543
           1       0.63      0.83      0.72       518

    accuracy                           0.84      2061
   macro avg       0.78      0.83      0.80      2061
weighted avg       0.86      0.84      0.84      2061



In [180]:
llama = Llama(
	model_path="../../models/Meta-Llama-3.1-8B-Instruct-Q4_K_M/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf",
    chat_format="llama-3",
    verbose=False
)
response = llama.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

In [181]:
predictions = []
skipped = []
test_cases = environmental_claims.text.to_list()[:300]
for idx, claim in tqdm(enumerate(test_cases), total=len(test_cases)):
    try:
        predictions.append(classify_claim(llama, prompt_template, output_format=output_format, claim=claim))
    except Exception as e:
        print(e, )
        skipped.append(idx)

100%|██████████| 300/300 [15:34<00:00,  3.11s/it]


In [182]:
ground_truth = environmental_claims.label.to_list()[:len(test_cases)]
for skip_idx in reversed(skipped):
    ground_truth.pop(skip_idx)

predictions_arr = np.array([pred.label for pred in predictions]).astype(int)
print(classification_report(ground_truth, predictions_arr))

              precision    recall  f1-score   support

           0       0.97      0.80      0.88       220
           1       0.63      0.93      0.75        80

    accuracy                           0.84       300
   macro avg       0.80      0.86      0.81       300
weighted avg       0.88      0.84      0.84       300

